<a href="https://colab.research.google.com/github/YaraElwakeel/text-mining-projects/blob/main/assignment_3_task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install necessary modules
!pip install datasets
!pip install mteb
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.

In [ ]:
import torch
from torch import nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader, TensorDataset
from datasets import load_dataset
import torch.nn as nn
from transformers import BertModel
import math
from tqdm import tqdm

### Custom Embedder

In [ ]:
# Load the AllNLI dataset
dataset = load_dataset("snli")

# filter out all the rows where label -1 (no class assigned)
dataset = dataset.filter(
    lambda x: 0 if x['label'] == -1 else 1
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Select only 6000 samples for training because of computation limitation
train_dataset = dataset['train'].shuffle(seed=42).select(range(6000))
validation_dataset = dataset['validation'].shuffle(seed=42).select(range(3000))
test_dataset = dataset['test'].shuffle(seed=42).select(range(3000))
device

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/550152 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/550152 [00:00<?, ? examples/s]

device(type='cuda')

In [ ]:
# using the pretrained bert tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to tokenize the dataset
def tokenize(batch):
    return tokenizer(batch['premise'], batch['hypothesis'], padding='max_length', truncation=True, max_length=512)

# Tokenize the datasets and add the tokenized columns
train_dataset = train_dataset.map(tokenize, batched=True)
validation_dataset = validation_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Convert labels to tensor
def convert_labels(batch):
    batch['label'] = 0 if batch['label'] == -1 else batch['label']
    batch['label'] = torch.tensor(batch['label']).long().to(device)
    return batch

train_dataset = train_dataset.map(convert_labels)
validation_dataset = validation_dataset.map(convert_labels)
test_dataset = test_dataset.map(convert_labels)

# Prepare the DataLoader
def prepare_dataloader(dataset, batch_size=8):
    input_ids = torch.stack([torch.tensor(x).to(device) for x in dataset['input_ids']])
    attention_mask = torch.stack([torch.tensor(x).to(device) for x in dataset['attention_mask']])
    labels = torch.tensor(dataset['label']).to(device)  # Move labels to device
    data = TensorDataset(input_ids, attention_mask, labels)
    return DataLoader(data, batch_size=batch_size)

train_loader = prepare_dataloader(train_dataset)
validation_loader = prepare_dataloader(validation_dataset)
test_loader = prepare_dataloader(test_dataset)

# test some data
for batch in train_loader:
    input_ids, attention_mask, labels = batch
    print("Input IDs:", input_ids)
    print("Attention Mask:", attention_mask)
    print("Labels:", labels)
    break


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Input IDs: tensor([[ 101, 1037, 2177,  ...,    0,    0,    0],
        [ 101, 1037, 2158,  ...,    0,    0,    0],
        [ 101, 2023, 3124,  ...,    0,    0,    0],
        ...,
        [ 101, 2093, 2111,  ...,    0,    0,    0],
        [ 101, 1037, 2879,  ...,    0,    0,    0],
        [ 101, 7739, 1998,  ...,    0,    0,    0]], device='cuda:0')
Attention Mask: tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')
Labels: tensor([0, 1, 0, 0, 0, 0, 1, 1], device='cuda:0')


In [ ]:
class EncoderEmbedder(nn.Module):
    def __init__(self, vocab_size, model_dim, nhead, num_encoder_layers, num_classes, max_seq_length=512, dropout=0.1):
        super(EncoderEmbedder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, model_dim)
        self.pos_encoder = nn.Parameter(torch.zeros(1, max_seq_length, model_dim))  # Positional encoding
        encoder_layers = nn.TransformerEncoderLayer(d_model=model_dim, nhead=nhead, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_encoder_layers)
        self.fc = nn.Linear(model_dim, num_classes)
        self.dropout = nn.Dropout(dropout)
        self.max_seq_length = max_seq_length

    def forward(self, input_ids, attention_mask):
        embedded = self.embedding(input_ids) * math.sqrt(self.embedding.embedding_dim)
        seq_length = embedded.size(1)
        embedded += self.pos_encoder[:, :seq_length, :]
        embedded = embedded.permute(1, 0, 2)  # Required shape (sequence, batch, model_dim)
        src_key_padding_mask = ~attention_mask.bool()  # This inverts the mask: True becomes False and vice versa
        transformer_out = self.transformer_encoder(embedded, src_key_padding_mask=src_key_padding_mask)
        transformer_out = transformer_out.mean(dim=0)  # Global average pooling
        output = self.fc(self.dropout(transformer_out))
        return output

    def encode(self, sentences: list[str], **kwargs) -> torch.Tensor:
        """Encodes the given sentences using the encoder."""

        inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt', max_length=self.max_seq_length)
        input_ids = inputs['input_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)

        with torch.no_grad():
            embedded = self.embedding(input_ids) * math.sqrt(self.embedding.embedding_dim)
            seq_length = embedded.size(1)
            embedded += self.pos_encoder[:, :seq_length, :]
            embedded = embedded.permute(1, 0, 2)
            src_key_padding_mask = ~attention_mask.bool()
            transformer_out = self.transformer_encoder(embedded, src_key_padding_mask=src_key_padding_mask)
            # Taking the mean across the sequence dimension to get fixed size embeddings.
            embeddings = transformer_out.mean(dim=0)

        return embeddings


# Hyperparameters
vocab_size = tokenizer.vocab_size
# embedding dimesion - kept to 16 because of computation limitation
model_dim = 128
# number of attention heads kept to 4 because of computation limitation
nhead = 4
# number of encoding layers kept to 3 because of computation limitation
num_encoder_layers = 3
# 3 classes in the case of snli database - entailment, contradiction, and neutral
num_classes = 3
max_seq_length = 512


#initialize the model
model = EncoderEmbedder(vocab_size, model_dim, nhead, num_encoder_layers, num_classes, max_seq_length)
model = model.to(device)

# Training setup
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
loss_fn = nn.CrossEntropyLoss()

# Training loop
def train_model(model, train_loader, validation_loader, optimizer, loss_fn, epochs=3):
    for epoch in range(epochs):
        model.train()
        train_accuracy = 0
        train_loss = 0

        # training
        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1} Training")
        for batch in train_pbar:
            input_ids, attention_mask, labels = batch
            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            preds = torch.argmax(outputs, dim=1)
            train_accuracy += (preds == labels).sum().item()
            train_loss += loss.item()
            # Update the progress bar
            train_pbar.set_postfix(loss=f"{loss.item():.4f}", accuracy=f"{(preds == labels).float().mean().item():.4f}")

        train_accuracy /= len(train_loader.dataset)
        train_loss /= len(train_loader)

        # Validation
        model.eval()
        val_loss = 0
        val_accuracy = 0

        val_pbar = tqdm(validation_loader, desc=f"Epoch {epoch+1} Validation")
        with torch.no_grad():
            for batch in val_pbar:
                input_ids, attention_mask, labels = batch
                outputs = model(input_ids, attention_mask)
                loss = loss_fn(outputs, labels)
                val_loss += loss.item()
                preds = torch.argmax(outputs, dim=1)
                val_accuracy += (preds == labels).sum().item()
                val_pbar.set_postfix(loss=f"{loss.item():.4f}", accuracy=f"{(preds == labels).float().mean().item():.4f}")

        val_accuracy /= len(validation_loader.dataset)
        val_loss /= len(validation_loader)

        # Print epoch summary
        print(f"\nEpoch {epoch + 1} Summary: Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [ ]:
# Train the model
train_model(model, train_loader, validation_loader, optimizer, loss_fn)

Epoch 1 Validation: 100%|██████████| 375/375 [00:03<00:00, 104.80it/s, accuracy=0.2500, loss=1.0913]



Epoch 1 Summary: Train Loss: 1.1130, Train Accuracy: 0.3477, Validation Loss: 1.1032, Validation Accuracy: 0.3633


Epoch 2 Validation: 100%|██████████| 375/375 [00:03<00:00, 102.13it/s, accuracy=0.2500, loss=1.0899]



Epoch 2 Summary: Train Loss: 1.0906, Train Accuracy: 0.3860, Validation Loss: 1.0966, Validation Accuracy: 0.3707


Epoch 3 Validation: 100%|██████████| 375/375 [00:03<00:00, 98.89it/s, accuracy=0.2500, loss=1.0883]


Epoch 3 Summary: Train Loss: 1.0775, Train Accuracy: 0.4118, Validation Loss: 1.0989, Validation Accuracy: 0.3820


In [ ]:
# save the trained model
torch.save(model.state_dict(), 'encoder_embedding_model.pth')

In [ ]:
# test the encoding function
model.encode(["this is good"]).shape

torch.Size([1, 128])

In [ ]:
from mteb import MTEB
from mteb.tasks import AILAStatutes

#model is the trained transformer model
model = model

# Initialize MTEB with the AILAStatutes task
evaluation = MTEB(tasks=[AILAStatutes()])
results = evaluation.run(model, output_folder=f"results/team09_transformer_embedder")

───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Retrieval

- AILAStatutes, p2p

In [ ]:
#check the scores
results[0].scores

{'test': [{'ndcg_at_1': 0.04,
   'ndcg_at_3': 0.04592,
   'ndcg_at_5': 0.05147,
   'ndcg_at_10': 0.07358,
   'ndcg_at_20': 0.11901,
   'ndcg_at_100': 0.33846,
   'ndcg_at_1000': 0.33846,
   'map_at_1': 0.008,
   'map_at_3': 0.017,
   'map_at_5': 0.02305,
   'map_at_10': 0.03137,
   'map_at_20': 0.0428,
   'map_at_100': 0.086,
   'map_at_1000': 0.086,
   'recall_at_1': 0.008,
   'recall_at_3': 0.029,
   'recall_at_5': 0.055,
   'recall_at_10': 0.09667,
   'recall_at_20': 0.21667,
   'recall_at_100': 1.0,
   'recall_at_1000': 1.0,
   'precision_at_1': 0.04,
   'precision_at_3': 0.04667,
   'precision_at_5': 0.052,
   'precision_at_10': 0.044,
   'precision_at_20': 0.045,
   'precision_at_100': 0.0434,
   'precision_at_1000': 0.00434,
   'mrr_at_1': 0.04,
   'mrr_at_3': 0.08333333333333331,
   'mrr_at_5': 0.11133333333333333,
   'mrr_at_10': 0.12523809523809523,
   'mrr_at_20': 0.14255057601497229,
   'mrr_at_100': 0.1527158940545197,
   'mrr_at_1000': 0.1527158940545197,
   'nauc_ndcg_at

In [ ]:
from mteb import MTEB
from mteb.tasks import ClimateFEVER

# model is the trained transformer model
model = model

# Initialize MTEB with the ClimateFEVER task
evaluation = MTEB(tasks=[ClimateFEVER()])
# Run evaluation
results_climate_fever = evaluation.run(model, corpus_chunk_size=5000)

───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Retrieval

- ClimateFEVER, s2p

In [ ]:
#check the scores
results_climate_fever[0].scores

### Pre-trained Bert Embedder

In [ ]:
class BertClassifier(nn.Module):
    def __init__(self, model_bert, nnum_classes, max_seq_length=512):
        super(BertClassifier, self).__init__()
        self.bert = model_bert
        self.classifier = nn.Linear(model_bert.config.hidden_size, num_classes)  # num_classes classes for SNLI
        self.max_seq_length = max_seq_length

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs[1]  # CLS token output
        logits = self.classifier(cls_output)
        return logits

    def encode(self, sentences: list[str], **kwargs) -> torch.Tensor:
        """Encodes the given sentences using the encoder."""
        # Tokenization and data preparation here.
        print("hello", len(sentences))
        inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt', max_length=self.max_seq_length)
        input_ids = inputs['input_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)

        with torch.no_grad():
            outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state
            embeddings = embeddings.mean(1) #get the embedding matrix

        return embeddings


# Hyperparameters
num_classes = 3
max_seq_length = 512

model_bert = BertModel.from_pretrained('bert-base-uncased')
model_bert = BertClassifier(model_bert, num_classes,  max_seq_length)
model_bert = model_bert.to(device)

# Training setup
optimizer = torch.optim.Adam(model_bert.parameters(), lr=2e-5)
loss_fn = nn.CrossEntropyLoss()

# Training loop
def train_model(model_bert, train_loader, validation_loader, optimizer, loss_fn, epochs=3):
    for epoch in range(epochs):
        model_bert.train()
        train_accuracy = 0
        train_loss = 0
        # Progress bar for training
        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1} Training")
        for batch in train_pbar:
            input_ids, attention_mask, labels = batch
            optimizer.zero_grad()
            outputs = model_bert(input_ids, attention_mask)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            preds = torch.argmax(outputs, dim=1)
            train_accuracy += (preds == labels).sum().item()
            train_loss += loss.item()
            # Update the progress bar
            train_pbar.set_postfix(loss=f"{loss.item():.4f}", accuracy=f"{(preds == labels).float().mean().item():.4f}")

        train_accuracy /= len(train_loader.dataset)
        train_loss /= len(train_loader)

        # Validation
        model_bert.eval()
        val_loss = 0
        val_accuracy = 0
        # Progress bar for validation
        val_pbar = tqdm(validation_loader, desc=f"Epoch {epoch+1} Validation")
        with torch.no_grad():
            for batch in val_pbar:
                input_ids, attention_mask, labels = batch
                outputs = model_bert(input_ids, attention_mask)
                loss = loss_fn(outputs, labels)
                val_loss += loss.item()
                preds = torch.argmax(outputs, dim=1)
                val_accuracy += (preds == labels).sum().item()
                # Update the progress bar
                val_pbar.set_postfix(loss=f"{loss.item():.4f}", accuracy=f"{(preds == labels).float().mean().item():.4f}")

        val_accuracy /= len(validation_loader.dataset)
        val_loss /= len(validation_loader)

        # Print epoch summary
        print(f"\nEpoch {epoch + 1} Summary: Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

# Train the model
train_model(model_bert, train_loader, validation_loader, optimizer, loss_fn)


Epoch 1 Validation: 100%|██████████| 375/375 [01:22<00:00,  4.56it/s, accuracy=0.8750, loss=0.4374]



Epoch 1 Summary: Train Loss: 0.8478, Train Accuracy: 0.6000, Validation Loss: 0.6203, Validation Accuracy: 0.7490


Epoch 2 Validation: 100%|██████████| 375/375 [01:22<00:00,  4.55it/s, accuracy=0.8750, loss=0.3135]



Epoch 2 Summary: Train Loss: 0.5123, Train Accuracy: 0.8035, Validation Loss: 0.5842, Validation Accuracy: 0.7737


Epoch 3 Validation: 100%|██████████| 375/375 [01:22<00:00,  4.56it/s, accuracy=0.8750, loss=0.5143]


Epoch 3 Summary: Train Loss: 0.3078, Train Accuracy: 0.8927, Validation Loss: 0.7132, Validation Accuracy: 0.7757


In [ ]:
# test the encode function
model_bert.encode(["this is test sentence"]).shape

hello 1


torch.Size([1, 768])

In [ ]:
# save the bert model
torch.save(model.state_dict(), 'bert_embedding_model.pth')

In [ ]:
from mteb import MTEB
from mteb.tasks import AILAStatutes

# model_bert trained model
model = model_bert

# Initialize MTEB with the AILAStatutes task
evaluation = MTEB(tasks=[AILAStatutes()])
results_bert = evaluation.run(model, output_folder=f"results/team09_bert_embedder")

In [ ]:
#check the score
results_bert[0].scores

In [ ]:
from mteb import MTEB
from mteb.tasks import ClimateFEVER

# model_bert trained model
model = model_bert

# Initialize MTEB with the ClimateFEVER task
evaluation = MTEB(tasks=[ClimateFEVER()])
# Run evaluation
results_climate_fever_bert = evaluation.run(model, output_folder=f"results/climate_fever_team09_bert_model_embedder")

In [ ]:
#check the score
results_climate_fever_bert[0].scores

In [ ]:
# extra work (ignore)
from mteb import MTEB
from mteb.tasks import AILAStatutes
from sentence_transformers import SentenceTransformer

model_name = "all-MiniLM-L12-v2"
model = SentenceTransformer(model_name)

evaluation = MTEB(tasks=[AILAStatutes()])
results = evaluation.run(model, output_folder=f"results/{model_name}gte-Qwen2-7B-instruct")

In [ ]:
# extra work (ignore)
results[0].scores

In [ ]:
# extra work (ignore)
from mteb import MTEB
from mteb.tasks import ClimateFEVER
from sentence_transformers import SentenceTransformer

model_name = "all-MiniLM-L12-v2"
model = SentenceTransformer(model_name)

evaluation = MTEB(tasks=[ClimateFEVER()])
results_climate_fever_bert = evaluation.run(model, output_folder=f"results/{model_name}", corpus_chunk_size=10)

In [ ]:
# extra work (ignore)
results_climate_fever_bert[0].scores

### Results

In [ ]:
import pandas as pd

score_encoder_embedder = 0 # TODO: add the nDCG@10 for the custom model
score_bert_embedder = 0 # TODO: add the nDCG@10 for the custom model

# Creating a DataFrame to represent the table in the image
data = {
    "Model": ["Anastasia's Transformer", "bert-base-uncased"],
    "Dataset": ["SNLI", "SNLI"],
    "nDCG@10": [score_encoder_embedder, score_bert_embedder]
}

table_df = pd.DataFrame(data)
table_df


,Model,Dataset,nDCG@10
0,Anastasia's Transformer,SNLI,0
1,bert-base-uncased,SNLI,0
